In [1]:
!pip install contractions
!pip install emoji
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [14]:
# from google.colab import drive
# drive.mount('/content/gdrive')
import os
import pandas as pd
# os.chdir("/content/gdrive/My Drive/Colab Notebooks/DATA70202")
data_c = pd.read_csv('rr_comments.csv')
data_p = pd.read_csv('rr_posts.csv')
data_c['source'] = 'comment'
data_p['source'] = 'post'
#combine commnet  and post dataset
data_c = data_c.rename(columns={'comment_body': 'text'})
data = pd.concat([data_c[['text', 'source']], data_p[['text','source']]], ignore_index=True)
data.dropna(inplace=True)
data1=data['text'].astype(str).copy()
data1


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,text
0,"Perfect, initial matches we are good and Guwah..."
1,It's good to have the initial matches in Guwah...
2,"Just crush that fucking dc,\nLast year dc owne..."
3,Two Punjab games\nTwo RCB games \nTwo Gujarat ...
4,Apr9 match to not look forward to
...,...
252,I was part of a mock auction just now and I fe...
254,1. Sanju Samson\n2. Yashasvi Jaiswal\nA left-h...
255,1. Yashasvi Jaiswal\n2. Sanju Samson (C) (WK)\...
256,Jaiswal Parag Sanju Jurel Rana Hetty is a very...


In [37]:
import pandas as pd
import re
import contractions
import emoji
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
def clean_text(text):
    # Expand contractions
    expanded = contractions.fix(text)

    # Remove URLs and mentions/hashtags
    cleaned = re.sub(r"http\S+|www\S+|@\w+|#\w+", "", expanded)
    cleaned = re.sub(r'[\r\n]+', '', text)
    # Remove special characters but preserve emojis
    emoji_chars = set(emoji.EMOJI_DATA.keys())
    pattern = r"[^\w\s" + re.escape("".join(emoji_chars)) + "]"
    cleaned = re.sub(pattern, '', cleaned)

    # Normalize whitespace and lowercase
    cleaned = re.sub(r"\s+", " ", cleaned).strip().lower()
    return cleaned

def sentiment_analysis(input_data, text_col='text', output_file='results.csv'):
    # Unified input format handling
    if isinstance(input_data, str):
        df = pd.DataFrame({text_col: [input_data]})
    elif isinstance(input_data, pd.Series):
        df = input_data.to_frame(name=text_col)
    elif isinstance(input_data, pd.DataFrame):
        df = input_data.copy()
    else:
        raise ValueError("Input must be a string, Series, or DataFrame")

    # Text preprocessing
    df['cleaned_text'] = df[text_col].apply(clean_text)

    # Filter empty texts
    df = df[df['cleaned_text'].str.strip().ne('')]

    # VADER analysis
    vader = SentimentIntensityAnalyzer()
    vader_results = df['cleaned_text'].apply(vader.polarity_scores).apply(pd.Series)
    df = pd.concat([df, vader_results], axis=1)

    # Scale compound score to 0-1 range
    df['vader_scaled'] = (df['compound'] + 1) / 2

    # Identify samples for secondary verification
    df['needs_verify'] = (df['vader_scaled'] >= 0.3) & (df['vader_scaled'] <= 0.7)

    # Initialize results columns
    df['model_source'] = 'vader'
    df['final_sentiment'] = np.select(
        [
            df['vader_scaled'] >= 0.7,
            df['vader_scaled'] <= 0.3
        ],
        [
            'positive',
            'negative'
        ],
        default='neutral'
    )

    # RoBERTa model for verification
    roberta_pipe = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment",
        device=-1,
        truncation=True,
        padding=True,
        top_k=None,
        max_length=512
    )

    if df['needs_verify'].any():
        mask = df['needs_verify']
        texts_to_verify = df.loc[mask, 'cleaned_text'].tolist()
        try:
            batch_results = roberta_pipe(texts_to_verify, batch_size=8)
        except Exception as e:
            print(f"Error during RoBERTa processing: {e}")
            batch_results = []

        roberta_scores = []
        for result in batch_results:
            try:
                score_dict = {item['label']: item['score'] for item in result}
                combined = (score_dict['LABEL_2'] - score_dict['LABEL_0'] + 1) / 2
                roberta_scores.append(combined)
            except KeyError:
                roberta_scores.append(np.nan)

        df.loc[mask, 'roberta_score'] = roberta_scores
        df['final_score'] = df['vader_scaled']
        df.loc[mask, 'final_score'] = 0.4*df['vader_scaled'] + 0.6*df['roberta_score']

        # final sentiment results
        conditions = [
            df['final_score'] >= 0.6,
            df['final_score'] <= 0.4,
            df['final_score'].isna()
        ]
        choices = ['positive', 'negative', 'neutral']
        df['final_sentiment'] = np.select(conditions, choices, default='neutral')

        # record model source
        df.loc[mask, 'model_source'] = 'hybrid'

    df.to_csv(output_file, index=False)
    print(f"Analysis complete. Results saved to: {output_file}")
    return df

In [38]:
if __name__ == "__main__":

    results = sentiment_analysis(data1)

    print(results[['cleaned_text', 'final_sentiment', 'model_source']])

Device set to use cpu


Analysis complete. Results saved to: results.csv
                                          cleaned_text final_sentiment  \
0    perfect initial matches we are good and guwaha...        positive   
1    its good to have the initial matches in guwaha...        positive   
2    just crush that fucking dclast year dc owner c...        negative   
3    two punjab gamestwo rcb games two gujarat game...        positive   
4                    apr9 match to not look forward to        negative   
..                                                 ...             ...   
252  i was part of a mock auction just now and i fe...        positive   
254  1 sanju samson2 yashasvi jaiswala lefthand and...        negative   
255  1 yashasvi jaiswal2 sanju samson c wk3 glenn m...        positive   
256  jaiswal parag sanju jurel rana hetty is a very...        negative   
257  seeing buttler in the recent t20 seriesi mean ...        positive   

    model_source  
0          vader  
1          vader  
2    

In [42]:
results

,text,cleaned_text,neg,neu,pos,compound,vader_scaled,needs_verify,model_source,final_sentiment,roberta_score,final_score
0,"Perfect, initial matches we are good and Guwah...",perfect initial matches we are good and guwaha...,0.000,0.680,0.320,0.7650,0.88250,False,vader,positive,NaN,0.882500
1,It's good to have the initial matches in Guwah...,its good to have the initial matches in guwaha...,0.102,0.601,0.297,0.9320,0.96600,False,vader,positive,NaN,0.966000
2,"Just crush that fucking dc,\nLast year dc owne...",just crush that fucking dclast year dc owner c...,0.082,0.918,0.000,-0.1531,0.42345,True,hybrid,negative,0.017890,0.180114
3,Two Punjab games\nTwo RCB games \nTwo Gujarat ...,two punjab gamestwo rcb games two gujarat game...,0.000,1.000,0.000,0.0000,0.50000,True,hybrid,positive,0.698891,0.619335
4,Apr9 match to not look forward to,apr9 match to not look forward to,0.000,1.000,0.000,0.0000,0.50000,True,hybrid,negative,0.128245,0.276947
...,...,...,...,...,...,...,...,...,...,...,...,...
252,I was part of a mock auction just now and I fe...,i was part of a mock auction just now and i fe...,0.094,0.749,0.157,0.8754,0.93770,False,vader,positive,NaN,0.937700
254,1. Sanju Samson\n2. Yashasvi Jaiswal\nA left-h...,1 sanju samson2 yashasvi jaiswala lefthand and...,0.098,0.819,0.083,-0.8836,0.05820,False,vader,negative,NaN,0.058200
255,1. Yashasvi Jaiswal\n2. Sanju Samson (C) (WK)\...,1 yashasvi jaiswal2 sanju samson c wk3 glenn m...,0.072,0.781,0.147,0.9810,0.99050,False,vader,positive,NaN,0.990500
256,Jaiswal Parag Sanju Jurel Rana Hetty is a very...,jaiswal parag sanju jurel rana hetty is a very...,0.067,0.906,0.027,-0.7405,0.12975,False,vader,negative,NaN,0.129750


In [39]:
results.value_counts('model_source')

,count
model_source,
vader,104
hybrid,98


In [40]:
results.value_counts('final_sentiment')

,count
final_sentiment,
positive,95
negative,54
neutral,53
